In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Make sure your codes are properly documented. We recommend the following:-
a. Before each code block have a markdown block which mentions the following
i. What the code block is doing.
ii. What is your intuition behind doing this? Why do you think it is useful?

b. For bigger code blocks also add comments in the block.
c. Keep everything in this notebook things which worked, things which did not work.
This notebook should be a snapshot of the process you follow to solve a Data
problem.

In [27]:
import tweepy
import jsonlines

In [3]:
consumer_key = ''  
consumer_secret = ''
access_token = ''
access_token_secret = ''

In [6]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import time

In [33]:
def get_all_tweets(screen_name):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    tweets_data = []
    for status in tweepy.Cursor(api.user_timeline, screen_name=screen_name).items():
        tweets_data.append(status)
    return tweets_data

In [34]:
tweets_data = get_all_tweets("midasIIITD")

In [45]:
with jsonlines.open('tweets_data.jsonl', 'w') as writer:
    for tweet in tweets_data:
        writer.write(tweet._json)